## Оконные(аналитические) функции

Оконные функции - полезный инструмент для построения сложных аналитических запросов.

Для их использования нужно задать парметры окна и функцию, которую хотим посчитать на каждом объекте внутри окна.

Простой пример - функция ROW_NUMBER(). Эта функция нумерует строки внутри окна. Пронумеруем контент для каждого пользователя в порядке убывания рейтингов.

<pre>
SELECT
  userId, movieId, rating,
  ROW_NUMBER() OVER (PARTITION BY userId ORDER BY rating DESC) as movie_rank
FROM (
    SELECT DISTINCT
        userId, movieId, rating
    FROM ratings
    WHERE userId <>1 LIMIT 1000
) as sample
ORDER BY
    userId,
    rating DESC,
    movie_rank
LIMIT 20;
</pre>

Результат:
<pre>
userid | movieid | rating | movie_rank
--------+---------+--------+------------
      2 |    1356 |      5 |          1
      2 |     339 |      5 |          2
      2 |     648 |      4 |          3
      2 |     605 |      4 |          4
      2 |    1233 |      4 |          5
      2 |    1210 |      4 |          6
      2 |     377 |      4 |          7
      2 |     260 |      4 |          8
      2 |      79 |      4 |          9
      2 |     628 |      4 |         10
      2 |      64 |      4 |         11
      2 |      58 |      3 |         12
      2 |      25 |      3 |         13
      2 |     762 |      3 |         14
</pre>

Параметры запроса:

* ROW_NUMBER - функция, которую применяем к окну
* OVER - описание окна

Описание окна содержит:
* PARTITION BY - поле (или список полей), которые описывают группу строк для применения оконной функции
* ORDER BY - поле, которое задаёт порядок записей внутри окна. Для полей внутри ORDER BY можно применять стандартные модификаторы DESC, ASC

Оконнная функция никак не меняет количество строк в выдаче, но к каждой строке добавляется полезная информация - например, про порядковый номер строки внутри окна

Названия функций обычно отражают их ссмысл. Ниже будут прриведены примеры использования и результаты запросо

### SUM()

Суммирует значения внутри окна. Посчитаем странную метрику - разделим каждое значение рейтинга на сумму всех рейтингов этого пользователя.

<pre>
SELECT userId, movieId, rating,
    rating / SUM(rating) OVER (PARTITION BY userId) as strange_rating_metric
FROM (SELECT DISTINCT userId, movieId, rating FROM ratings WHERE userId <>1 LIMIT 1000) as sample
ORDER BY userId, rating DESC
LIMIT 20;
</pre>

Результат:
<pre>
 userid | movieid | rating | strange_rating_metric
--------+---------+--------+-----------------------
      2 |     339 |      5 |    0.0684931506849315
      2 |    1356 |      5 |    0.0684931506849315
      2 |     648 |      4 |    0.0547945205479452
      2 |      64 |      4 |    0.0547945205479452
      2 |      79 |      4 |    0.0547945205479452
      2 |     260 |      4 |    0.0547945205479452
      2 |    1233 |      4 |    0.0547945205479452
      2 |    1210 |      4 |    0.0547945205479452
      2 |     377 |      4 |    0.0547945205479452
      2 |     605 |      4 |    0.0547945205479452
      2 |     628 |      4 |    0.0547945205479452
      2 |     762 |      3 |    0.0410958904109589
      2 |     141 |      3 |    0.0410958904109589
      2 |     780 |      3 |    0.0410958904109589
      2 |       5 |      3 |    0.0410958904109589
      2 |      58 |      3 |    0.0410958904109589
      2 |      25 |      3 |    0.0410958904109589
      2 |    1475 |      3 |    0.0410958904109589
      2 |      32 |      2 |    0.0273972602739726
      2 |    1552 |      2 |    0.0273972602739726
(20 rows)
</pre>

### COUNT(), AVG()

Счётчик элементов внутри окна, а так же функция Average(). Для наглядности воспользуемся ими одновременно - результаты не должны отличаться.
Вычислим полезную метрику - отклонение рейтинга пользователя от среднего рейтинга, который он склонен выставлять

<pre>
SELECT userId, movieId, rating,
    rating - AVG(rating) OVER (PARTITION BY userId) rating_deviance_simplex,
    rating - SUM(rating) OVER (PARTITION BY userId) /COUNT(rating) OVER (PARTITION BY userId) as rating_deviance_complex
FROM (SELECT DISTINCT userId, movieId, rating FROM ratings WHERE userId <>1 LIMIT 1000) as sample
ORDER BY userId, rating DESC
LIMIT 20;
</pre>

Результат:
<pre>
 userid | movieid | rating | rating_deviance_simplex | rating_deviance_complex
--------+---------+--------+-------------------------+-------------------------
      2 |     339 |      5 |        1.68181818181818 |        1.68181818181818
      2 |    1356 |      5 |        1.68181818181818 |        1.68181818181818
      2 |     648 |      4 |       0.681818181818182 |       0.681818181818182
      2 |      64 |      4 |       0.681818181818182 |       0.681818181818182
      2 |      79 |      4 |       0.681818181818182 |       0.681818181818182
      2 |     260 |      4 |       0.681818181818182 |       0.681818181818182
      2 |    1233 |      4 |       0.681818181818182 |       0.681818181818182
      2 |    1210 |      4 |       0.681818181818182 |       0.681818181818182
      2 |     377 |      4 |       0.681818181818182 |       0.681818181818182
      2 |     605 |      4 |       0.681818181818182 |       0.681818181818182
      2 |     628 |      4 |       0.681818181818182 |       0.681818181818182

</pre>

## Практика 1
Выведите таблицу с 3-мя полями: название фильма, имя актера и количества фильмов, в которых он снимался

## PostgreSQL CTE

CTE - это временный результат запроса, который можно использовать с другими запросами. Временный = существует только в рамках запроса.

Синтаксис:

```sql
WITH cte_name (column_list) AS (
    CTE_query_definition 
)
statement;
```

- указывается название cte
- опционально список имен колонок 
- запрос cte
- основной sql запрос

Обычно используются для упрощения сложных join-запросов и подзапросов. Кроме того поддерживают рекурсинвые запросы

```sql
WITH cte_film AS (
    SELECT 
        film_id, 
        title,
        (CASE 
            WHEN length < 30 THEN 'Short'
            WHEN length >= 30 AND length < 90 THEN 'Medium'
            WHEN length > 90 THEN 'Long'
        END) length    
    FROM
        film
)
SELECT
    film_id,
    title,
    length
FROM 
    cte_film
WHERE
    length = 'Long'
ORDER BY 
    title;
```

## Практика 2

При помощи CTE выведите таблицу со следующим содержанием Фамилия и Имя сотрудника (staff) и количество прокатов двд (retal), которые он продал

## Сложные типы данных

### JSON

JSON - JavaScript Object Notation. JSON - де-факто стандарт для хранения данных в виде key-value пар. 

Рассмотрим простой пример:

```sql
CREATE TABLE orders (
 ID serial NOT NULL PRIMARY KEY,
 info json NOT NULL
);

INSERT INTO orders (info)
VALUES
 (
 '{ "customer": "John Doe", "items": {"product": "Beer","qty": 6}}'
 ),
 (
 '{ "customer": "Lily Bush", "items": {"product": "Diaper","qty": 24}}'
 ),
 (
 '{ "customer": "Josh William", "items": {"product": "Toy Car","qty": 1}}'
 ),
 (
 '{ "customer": "Mary Clark", "items": {"product": "Toy Train","qty": 2}}'
 );
```

Запрос данных:

```sql
SELECT
 info
FROM
 orders;
```

Postgres вовзращает ответ в виде типа JSON. Для работы с ним есть 2 специальных оператора -> и ->>

-> возвращает результат в виде JSON-объекта
->> вовзращает результат в виде текста

Получить имена всех покупателей:
```sql
SELECT
 info ->> 'customer' AS customer
FROM
 orders;
```

т.к. оператор -> возвращает JSON-объект, то к его результату можно снова применять оператор -> и ->>. Пример:

``` sql
SELECT
 info -> 'items' ->> 'product' as product
FROM
 orders
ORDER BY
 product;
```

Еще пример. Поиск людей, которые купили 2 продукта:

```sql
SELECT
 info ->> 'customer' AS customer,
 info -> 'items' ->> 'product' AS product
FROM
 orders
WHERE
 CAST (
 info -> 'items' ->> 'qty' AS INTEGER
 ) = 2
```

## Практика 3

Создайте таблицу orders (скрипт выше в лекции). Выведите общее количество заказов

## Массивы

Массив - это коллекция элементов. В одной колонке Вы можете хранить несколько аттрибутов одного типа. Пример:

```sql
CREATE TABLE contacts (
 id serial PRIMARY KEY,
 name VARCHAR (100),
 phones TEXT []
);
```

### Вставка данных:
```sql
INSERT INTO contacts (name, phones)
VALUES
 (
 'John Doe',
 ARRAY [ '(408)-589-5846',
 '(408)-589-5555' ]
 );
```

или


```sql
INSERT INTO contacts (name, phones)
VALUES
 (
 'John Doe',
 '{ "(408)-589-5846", "(408)-589-5555" }'
 );
```

### Выборка данных:

```sql
SELECT
 name,
 phones
FROM
 contacts;
```

### Запрос конкретного элемента массива:

```sql
SELECT
 name,
 phones [ 1 ]
FROM
 contacts;
```

Индексы начинаются с 1.

###  Модификация данных

Обновление элемента
```sql
UPDATE contacts
SET phones [ 2 ] = '(408)-589-5843'
WHERE
 ID = 3;
```

Обновдение всего массива
```sql
UPDATE contacts
SET phones = '{"(408)-589-5843"}'
WHERE
 ID = 3;
```

### Поиск элемента

```sql 
SELECT
 name,
 phones
FROM
 contacts
WHERE
 '(408)-589-5555' = ANY (phones);
```

### Expand

unnest - превращает массив в набор строк

```sql
SELECT
 name,
 unnest(phones)
FROM
 contacts;
```

## Практика 4

Выведите сколько раз встречается какой специлаьный аттрибут (special_feature) у фильма 

## Домашнее задание
1. Сделайте запрос к таблице rental. Добавьте колонку с порядковым номером аренды (сортировать по rental_date) для каждого юзера.
2. Для каждого пользователя подсчитайте сколько он брал в аренду фильмов со специальным аттрибутом Behind the Scenes

## Дополнительные материалы:

- https://habr.com/ru/post/269497/
- https://medium.com/@hakibenita/be-careful-with-cte-in-postgresql-fca5e24d2119
- https://postgrespro.ru/docs/postgrespro/10/datatype-json